### Exercises

#### Question 1

Alongside this notebook is a data file named `daily_quotes.csv` which contains EOD OHLC/Volume data for a small number of equities over a 6 month period.

The first step is to load up this data into a dataframe, ensuring that all data types are correct (datetime objects for dates, floats for OHLC data, and integers for Volume).

Write a function that receives the file name as an argument and returns a dataframe that:
- has the correct data type for each column (`str`, `float`, `int`)
- has a row index based on the `symbol` column

In addition, we would like our dataframe to contain columns named and ordered in a specific way:
- symbol (`str`)
- date (`datetime`)
- open (`float`)
- high (`float`)
- low (`float`)
- close (`float`)
- volume (`int`)

(with `symbol` being used as the row index)

Hint: 

You will want to read up the Pandas docs for `read_csv` to see how you can handle datetime data directly while loading the data (in particular you should look at the `parse_dates` option):

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html

Alternatively, you could convert these objects into proper datetime types after loading by using the Pandas function `to_datetime`, documented here:

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.to_datetime.html

and then use conatenation to build up a dataframe that replaces the "old" `date` column with the "new" (properly typed) column.

#### Question 2

Write a function that, given a dataframe sructured as the one we created in Question 1 and a symbol name as a string (e.g. `AAPL`, `MSFT`, etc), will:
- return a similarly structured dataframe consisting of the row (or rows) containing the records with the highest volume for the given symbol
- raises a `ValueError` if the symbol is not in the dataframe

#### Question 3

Using the same dataframe as in the preceding questions, our goal now is to write a function that will return, for a specific symbol, the row that had the largest high-low spread.

Write a function to do that - it should just return a dataframe with the row (or rows) with the largest high-low spread.

#### Question 4

Using the same dataframe as the preceding questions, write a function that returns a single dataframe containing the record(s) with maximum high-low spread for each symbol in the dataframe. (Do not hardcode symbol names in this function - instead you should recover the possible symbol names from the data itself).

The returned dataframe should have the same structure as the original dataframe, but just contain the rows of maximum high-low spread for each symbol.

In [1]:
import pandas as pd

In [48]:
def load_df(url):
    df = pd.read_csv(
        url,
        header=0, 
        names=["symbol", "date","close", "volume", "open", "high", "low"], 
        index_col=0, 
        parse_dates=True)
    df = df[["date", "open", "high", "low", "close", "volume"]]
    return df


def max_volume(df, symbol):
    try:
        df_sub = df.loc[symbol, :]
    except KeyError:
        raise ValueError(f"Symbol {symbol} has no data")   
    return df_sub[df_sub["volume"]==df_sub["volume"].max()]


def max_spread(df, symbol):
    try:
        df_sub = df.loc[symbol, :]
    except KeyError:
        raise ValueError(f"Symbol {symbol} has no data")
    df_sub["spread"] = df_sub["high"]-df_sub["low"]  
    return df_sub[df_sub["spread"]==df_sub["spread"].max()]


def max_spreads(df): 
    return pd.concat([max_spread(df, s) for s in df.index.unique()], axis=0)


In [38]:
url = "https://raw.githubusercontent.com/anhailing/python-fundamentals/main/30%20-%20Pandas/11%20-%20Exercises/daily_quotes.csv"
df = load_df(url)

C:\Users\hahy\AppData\Local\Temp\ipykernel_14016\1831912288.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv(


In [39]:
max_volume(df, "AAPL")


,date,open,high,low,close,volume
symbol,,,,,,
AAPL,8/24/20,128.6975,128.785,123.9363,125.8575,345937760


In [40]:
max_volume(df, "MSFT")

,date,open,high,low,close,volume
symbol,,,,,,
MSFT,1/27/21,238.0,240.44,230.14,232.9,69870640


In [42]:
try:
    max_volume(df, "SPY")
except ValueError as e:
    print(e)

Symbol SPY has no data


In [46]:
max_spread(df, "AAPL")

,date,open,high,low,close,volume,spread
symbol,,,,,,,
AAPL,9/4/20,120.07,123.7,110.89,120.96,332607200,12.81


In [49]:
max_spreads(df)

,date,open,high,low,close,volume,spread
symbol,,,,,,,
AAPL,9/4/20,120.07,123.70,110.8900,120.96,332607200,12.8100
AMZN,9/4/20,3318.00,3381.50,3111.1300,3294.62,8781754,270.3700
GOOG,2/3/21,2073.00,2116.50,2018.3800,2070.07,4118170,98.1200
MSFT,9/3/20,229.27,229.31,214.9602,217.30,58400290,14.3498
